it's VERY likely that a document would only appear on SERPS from a single topic. That is, a document that appears for a user searching for Ethics would not appear for a user searching for ADHD symptoms. So here, we try to match these two and raise an error if we find a clash.

In [255]:
import json
import urllib.parse
from pprint import pprint
from tqdm.auto import tqdm
from wayback_api import get_from_wayback

dataset = json.load(open("../data/logs_with_position.json"))

In [276]:
if (
    answer.status_code != 200
    or len(answer.json()["archived_snapshots"]) == 0
    or not answer.json()["archived_snapshots"]["closest"]["available"]
):
    print("\tNot on archive!")

	Not on archive!


In [ ]:
url = m_url
wayback_url = f"http://archive.org/wayback/available?url={urllib.parse.quote(url)}&timestamp={timestamp}"
answer = requests.get(wayback_url)
if (
    answer.status_code != 200
    or len(answer.json()["archived_snapshots"]) == 0
    or not answer.json()["archived_snapshots"]["closest"]["available"]
):
    print("\tNot on archive!")

new_url = answer.json()["archived_snapshots"]["closest"]["url"]
http = urllib3.PoolManager(cert_reqs="CERT_REQUIRED", ca_certs=certifi.where())
page_content = http.request("GET", new_url)
start_comment = "<!-- BEGIN WAYBACK TOOLBAR INSERT -->"
end_comment = "<!-- END WAYBACK TOOLBAR INSERT -->"
page_content = page_content.data.decode().split(start_comment)[1].split(end_comment)[1].strip()

_, pars = clean_page(page_content)
" ".join(page_content)

KeyError: 'closest'

In [519]:
start_comment = "<!-- BEGIN WAYBACK TOOLBAR INSERT -->"
end_comment = "<!-- END WAYBACK TOOLBAR INSERT -->"

In [488]:
docs = {}
empty_docs = []
counter = 0
for idx, line in enumerate(open("../data/clicked_docs.tsv")):
    counter += 1
    try:
        url, text = line.strip().split(maxsplit=1)
    except ValueError:  # empty doc
        text = ""
        url = line.split()[0]
        empty_docs.append(url)
        # print(idx, url)
        continue
    docs[url] = text

In [530]:
for m_url in tqdm(empty_docs):
    if m_url in docs:
        continue
    # Try to re-fetch it again
    final_str = get_from_wayback(m_url)
    docs[m_url] = final_str

  0%|          | 0/241 [00:00<?, ?it/s]

trying to fetch https://quizlet.com/1377915/chapter-14-business-cycles-fluctuation-and-inflation-flash-cards/ at 20200826
http://web.archive.org/web/20150302102530/http://quizlet.com:80/1377915/chapter-14-business-cycles-fluctuation-and-inflation-flash-cards/
trying to fetch https://works.bepress.com/grace_lee/16/ at 20200826
	Not on archive! Trying today....
trying to fetch https://salud.ccm.net/faq/12955-gastroenterologia-definicion at 20200826
http://web.archive.org/web/20200815200048/https://salud.ccm.net/faq/12955-gastroenterologia-definicion
	 Could not get anyhing good from page...
trying to fetch https://www.scribd.com/document/223094684/Business-cycle-pdf at 20200826
	Not on archive! Trying today....
trying to fetch http://datingadviceac.us/radiocarbon-14-dating-wikipedia.vpn at 20200826
	Not on archive! Trying today....
trying to fetch https://quizlet.com/75842889/meta-ethics-what-is-meta-ethics-flash-cards/ at 20200826
	Not on archive! Trying today....
trying to fetch https:

In [535]:
# Doc->Topic mapping
doc_topic = {}
for u in dataset:
    topic = urllib.parse.quote(u["topic_title"])
    for c in u["clicks"]:
        url = c["url"]
        if "https://www.wikipedia.org/" in url:
            continue
        if url in docs and url in doc_topic and doc_topic[url] != topic:
            print(f"Ooops. url {url} found in two topics: {topic} and {doc_topic[url]}")
        doc_topic[url] = topic

No errors found! =)

In [536]:
# Write to file
with open("../data/clicked_docs_with_topics_more.tsv", "w") as outf:
    for url, text in docs.items():
        if "https://www.wikipedia.org/" in url:
            continue
        topic = doc_topic[url]
        outf.write(f"{url}\t{topic}\t{text}\n")